In [1]:
%matplotlib inline

In [2]:
################################################################
''' Preporcessing steps: 
1. lowercasing 
2. Digit -> DDD 
3. URLs -> httpAddress 
4. @username -> userID 
5. Remove special characters, keep ; . ! ? 
6. normalize elongation 
7. tokenization using tweetNLP
output is ~/Dropbox (QCRI)/AIDR-DA-ALT-SC/data/labeled datasets/prccd_data/{filename}_AIDR_prccd.csv
'''
#################################################################

#=================
#==> Libraries <==
#=================
import re, os
import string 
import sys
import twokenize
import csv
from collections import defaultdict
from os.path import basename
import ntpath
import codecs
import unicodedata

def process(lst):
    prccd_item_list=[]
    for tweet in lst:


#         # Normalizing utf8 formatting
#         tweet = tweet.decode("unicode-escape").encode("utf8").decode("utf8")
#         #tweet = tweet.encode("utf-8")
#         tweet = tweet.encode("ascii","ignore")
#         tweet = tweet.strip(' \t\n\r')

        # 1. Lowercasing
        tweet = tweet.lower()
        #print "[lowercase]", tweet

        # Word-Level
        tweet = re.sub(' +',' ',tweet) # replace multiple spaces with a single space

        # 2. Normalizing digits
        tweet_words = tweet.strip('\r').split(' ')
        for word in [word for word in tweet_words if word.isdigit()]:
            tweet = tweet.replace(word, "D" * len(word))
#         print( "[digits]", tweet)

        # 3. Normalizing URLs
        tweet_words = tweet.strip('\r').split(' ')
        for word in [word for word in tweet_words if '/' in word or '.' in word and  len(word) > 3]:
            tweet = tweet.replace(word, "")
#         print( "[URLs]", tweet)

        #4. Normalizing username

        tweet_words = tweet.strip('\r').split(' ')
        try:
            for word in [word for word in tweet_words if word[0] == '@' and len(word) > 1]:
                tweet = tweet.replace(word, "")
#         print( "[usrename]", tweet)
        except:
            tweet = tweet


        # 5. Removing special Characters
        punc = '@$%^&*()_+-={}[]:"|\'\~`<>/,'
        trans = str.maketrans(punc, ' '*len(punc))
        tweet = tweet.translate(trans)
        #print( "[punc]", tweet)

        # 6. Normalizing +2 elongated char
        tweet = re.sub(r"(.)\1\1+",r'\1\1', tweet)
        #print ("[elong]", tweet)

        # 7. tokenization using tweetNLP
        tweet = ' '.join(twokenize.simpleTokenize(tweet))
        #print( "[token]", tweet )

        #8. fix \n char
        tweet = tweet.replace('\n', ' ')

        prccd_item_list.append(tweet.strip())
#         print ("[processed]", tweet.replace('\n', ' '))
        
    return prccd_item_list


# Sample pipeline for text feature extraction and evaluation

The dataset used in this example is the 20 newsgroups dataset which will be
automatically downloaded and then cached and reused for the document
classification example.

You can adjust the number of categories by giving their names to the dataset
loader or setting them to None to get the 20 of them.

Here is a sample output of a run on a quad-core machine::

  Loading 20 newsgroups dataset for categories:
  ['alt.atheism', 'talk.religion.misc']
  1427 documents
  2 categories

  Performing grid search...
  pipeline: ['vect', 'tfidf', 'clf']
  parameters:
  {'clf__alpha': (1.0000000000000001e-05, 9.9999999999999995e-07),
   'clf__max_iter': (10, 50, 80),
   'clf__penalty': ('l2', 'elasticnet'),
   'tfidf__use_idf': (True, False),
   'vect__max_n': (1, 2),
   'vect__max_df': (0.5, 0.75, 1.0),
   'vect__max_features': (None, 5000, 10000, 50000)}
  done in 1737.030s

  Best score: 0.940
  Best parameters set:
      clf__alpha: 9.9999999999999995e-07
      clf__max_iter: 50
      clf__penalty: 'elasticnet'
      tfidf__use_idf: True
      vect__max_n: 2
      vect__max_df: 0.75
      vect__max_features: 50000


In [9]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Mathieu Blondel <mathieu@mblondel.org>
# License: BSD 3 clause
from pprint import pprint
from time import time
import pandas as pd
import os 

import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')
code_dir = os.getcwd()
parent_dir = os.path.dirname(code_dir)
print(parent_dir)
labled_data_folder  =  os.path.join(parent_dir,"Data/crisis_datasets_benchmarks/all_data_en")
initial_filtering_folder = os.path.join(parent_dir,"Data/crisis_datasets_benchmarks/initial_filtering")

# #############################################################################
# Load some categories from the training set
# categories = [
#     'alt.atheism',
#     'talk.religion.misc',
# ]
# Uncomment the following to do the analysis on all the categories
#categories = None

print("Loading data:")
# train = pd.read_table(os.path.join
#                        (labled_data_folder,
#                                     "crisis_consolidated_informativeness_filtered_lang_en_train.tsv"))
# test = pd.read_table(os.path.join
#                        (labled_data_folder,
#                                     "crisis_consolidated_informativeness_filtered_lang_en_test.tsv"),
#                        sep ='\t', quoting =3)
# dev = pd.read_table(os.path.join
#                      (labled_data_folder,
#                                   "crisis_consolidated_informativeness_filtered_lang_en_dev.tsv"))
# combinedf = pd.concat([train,test,dev])
# df_list = [combinedf,train, test, dev]
# df_list_name = ['combinedf','train', 'test', 'dev']
# proc_list = []
# for i, df in enumerate(df_list):
#     print("processing: "+df_list_name[i])
#     df['processed_txt'] = process(df['text'])
#     print("processing: "+df_list_name[i] + " complete")
    
# #############################################################################
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words = 'english')),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5,),
#     , 0.75, 1.0
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 2)),  # unigrams or bigrams
#     (1, 1), 
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
#     'tfidf__stop_words': ('english',),
    'clf__max_iter': (5,),
    'clf__alpha': (0.00001,),
#     , 0.000001
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(df_list[1]['processed_txt'], df_list[1]['processed_txt'])
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

 Preporcessing steps: 
1. lowercasing 
2. Digit -> DDD 
3. URLs -> httpAddress 
4. @username -> userID 
5. Remove special characters, keep ; . ! ? 
6. normalize elongation 
7. tokenization using tweetNLP
output is ~/Dropbox (QCRI)/AIDR-DA-ALT-SC/data/labeled datasets/prccd_data/{filename}_AIDR_prccd.csv

/Volumes/Elements/DataScience/dsa/capstone
Loading data:
Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (1e-05,),
 'clf__max_iter': (5,),
 'clf__penalty': ('l2', 'elasticnet'),
 'vect__max_df': (0.5,),
 'vect__ngram_range': (1, 2)}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan]
  warnings.warn(


TypeError: cannot unpack non-iterable int object